# Linear regression: three-factor model

$y = a + β_1x_1 + β_2x_2 + β_3x_3 + \varepsilon$
* y - dependent variable 
* α  - the intercept
* $x$1, $x$2, and $x$3  - independent variables
* β1, β2 and β3 - coefficients
* ε - random factor

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.stattools as tools
from scipy.stats import probplot

In [3]:
today = datetime.today()
days = timedelta(days=1000)
start = today-days

In [4]:
print('start:', start, '\t', 'end:', today)

start: 2019-10-28 19:52:38.022835 	 end: 2022-07-24 19:52:38.022835


In [ ]:
stk = 'EBAY'
stk = yf.download(stk, start, today)
y = stk['Adj Close']
x = stk[['Open', 'High', 'Volume']]
x=sm.add_constant(x)

In [6]:
result = sm.OLS(y,x).fit()

In [7]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              Adj Close   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 8.869e+04
Date:                Sun, 24 Jul 2022   Prob (F-statistic):               0.00
Time:                        19:52:38   Log-Likelihood:                -660.93
No. Observations:                 689   AIC:                             1330.
Df Residuals:                     685   BIC:                             1348.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0470      0.141     -0.334      0.7

# Risk and Performance Measures



Sharpe Ratio:

SR = $\frac{\overline{R}-\overline{R_f}}{\sigma}$ = $\frac{\overline{R}-\overline{R_f}}{\sqrt{var(\overline{R}-\overline{R_f})}}$
* SR - Sharpe Ratio
 * Annual Sharpe Ratio of 0-1 is condsidered sub-optimal
 * Annual Sharpe Ratio of 1-2 is condsidered good
 * Annual Sharpe Ratio of 2-3 is condsidered very good
 * Annual Sharpe Ratio above 3 is condsidered excellent
* $\overline{R}$ - mean return for a portfolio or a stock
* $\overline{R_f}$ - mean return for a risk-free security
* $\sigma$ - standard deviation (variance) of the portfolio or stock returnd

In [8]:
ret = pd.DataFrame()
ret['Daily Return'] = stk['Adj Close'].pct_change(1)
ret.dropna(inplace=True)
rf = 0.003/np.sqrt(252) # de-annualize risk-free rate by dividing by sqrt(252)

In [9]:
def compute_sharpe_ratio(data, risk_free_rate = 0):
    mean_return = data["Daily Return"].mean()
    std = data["Daily Return"].std()
    sharpe_ratio = (mean_return-risk_free_rate) / std
    return sharpe_ratio

In [10]:
# daily sharpe ratio with risk-free rate
sharp_ratio_daily_stk = compute_sharpe_ratio(ret, rf)

print(f"The daily sharpe ratio for this stock is {sharp_ratio_daily_stk}")

The daily sharpe ratio for this stock is 0.022234544284216316


In [11]:
# daily sharpe ratio w/o risk-free rate
daily_sharp_ratio = compute_sharpe_ratio(ret)
print(f"The daily sharpe ratio for this stock is {daily_sharp_ratio}")

The daily sharpe ratio for this stock is 0.030685211291844545


In [12]:
annual_sharp_ratio = np.sqrt(252) * daily_sharp_ratio
print(f"The yearly sharpe ratio for this stock is {annual_sharp_ratio}")

The yearly sharpe ratio for this stock is 0.4871126280341501


# Sortino Ratio

$S$ = $\frac{\overline{R}_i - T}{DR}$

$DR$ = $\sqrt{\int^{T}_{-\infty}(T-r)^2f(r)dr}$

 * $\overline{R}_i$ - mean  return for a portfolio or a stock

 * ${T}$ - target or required rate of return for the investment strategy under consideration
 * $DR$ - downside deviation or downside risk

 * $f(r)$ - the distribution for the annual returns

 * $r$ - random variable representing the return for the distribution of annual returns $f(r)$

In [13]:
def compute_sortino_ratio(data, target, risk_free_rate=0):
    mean_return = data["Daily Return"].mean()
    downside = data[data["Daily Return"] < target]["Daily Return"]
    std = downside.std()
    sortino_ratio = (mean_return-risk_free_rate) / std
    return sortino_ratio

In [14]:
sor_ratio = compute_sortino_ratio(ret, target=0)

print(f"The daily sortino ratio for this stock is {sor_ratio}")

The daily sortino ratio for this stock is 0.0428402521067778


In [15]:
annual_sor_ratio = np.sqrt(252) * sor_ratio
print(f"The yearly sortino ratio for this stock is {annual_sor_ratio}")

The yearly sortino ratio for this stock is 0.6800679190670698


## Probabilistic Sharpe Ratio

The problem of the sharpe ratio is, that it is calculated via historical data and thus it only yields an estimation and not the true sharpe ratio!

To tackle this problem you can use the *Probabilistic Sharpe Ratio* 
defined as 
$$ \text{cdf} \Big( \frac{(\widehat{\text{SR}} - SR^*)}{\widehat{\sigma}({\widehat{SR})}} \Big)$$
where cdf is the cummulative density function of the normal function, <br />
$\widehat{SR}$ is the original sharpe ratio,<br />
$SR^*$ is the benchmark sharpe ratio (often 0), <br />
and $\hat{\sigma}(\widehat{SR})$ is the standard deviation of the estimated sharpe ratio which is computed as follows:

$$ \hat{\sigma}(\hat{SR}) = \sqrt{\frac{1}{n-1} \Big (1 + \frac{1}{2} \widehat{\text{SR}}^2 - \gamma_3 \hat{\text{SR}} + \frac{\gamma_4}{4} \widehat{\text{SR}}^2 \Big )} $$

Here, $\gamma_3$ and $\gamma_4$ correspond to skew and fisher kurtosis.

In words, the probabilistic sharpe ratio computes the propability that the true sharpe ratio is $\leq$ the estimated sharpe ratio ($PSR = P(SR \leq \widehat{\text{SR}})$) given the benchmark sharpe ratio.

In [16]:
import scipy.stats

In [17]:
def compute_prob_sharpe_ratio(data, benchmark=0):
    
    sr = compute_sharpe_ratio(data, 0)
    
    skew = scipy.stats.skew(data["Daily Return"])
    # Use fisher kurtosis
    
    kurtosis = scipy.stats.kurtosis(data["Daily Return"], fisher=True)  
    
    n = len(data)
    
    std = ( (1 / (n-1)) * (1 + 0.5 * sr**2 - skew * sr + (kurtosis / 4) * sr**2))**0.5
    
    ratio = (sr - benchmark) / std
    prob_sharpe_ratio = scipy.stats.norm.cdf(ratio)
    return prob_sharpe_ratio

In [18]:
psr_stk = compute_prob_sharpe_ratio(ret)

print(f"The daily probabilistic sharpe ratio for this stock is {psr_stk}")

The daily probabilistic sharpe ratio for this stock is 0.7881642257157864


In [19]:
annual_psr = np.sqrt(252) * psr_stk
print(f"The yearly probabilistic sharpe ratio for this stock is {annual_psr}")

The yearly probabilistic sharpe ratio for this stock is 12.511719201130498
